Note that variant tags must be capitalized:

In [2]:
open Core

In [3]:
type basic_color =
   | Black | Red | Green | Yellow | Blue | Magenta | Cyan | White ;;

type basic_color =
    Black
  | Red
  | Green
  | Yellow
  | Blue
  | Magenta
  | Cyan
  | White


In [4]:
Cyan

- : basic_color = Cyan


In [9]:
[Yellow; Blue; Cyan]

- : basic_color list = [Yellow; Blue; Cyan]


In [10]:
let basic_color_to_int = function
  | Black -> 0 | Red     -> 1 | Green -> 2 | Yellow -> 3
  | Blue  -> 4 | Magenta -> 5 | Cyan  -> 6 

File "[10]", line 1, characters 25-133:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
White


val basic_color_to_int : basic_color -> int = <fun>


In [11]:
let basic_color_to_int = function
  | Black -> 0 | Red     -> 1 | Green -> 2 | Yellow -> 3
  | Blue  -> 4 | Magenta -> 5 | Cyan  -> 6 | White  -> 7 ;;

val basic_color_to_int : basic_color -> int = <fun>


In [14]:
let color_by_number number text =
    sprintf "\027[38;5;%dm%s\027[0m" number text;;

val color_by_number : int -> string -> string = <fun>


In [15]:
let blue = color_by_number (basic_color_to_int Blue) "Blue";;

val blue : string = "\027Blue\027"


In [17]:
printf "Hello %s World!\n" blue;;

- : unit = ()


In [18]:
type weight = Regular | Bold

type weight = Regular | Bold


In [19]:
  type color =
  | Basic of basic_color * weight (* basic colors, regular and bold *)
  | RGB   of int * int * int       (* 6x6x6 color cube *)
  | Gray  of int                   (* 24 grayscale levels *)

type color =
    Basic of basic_color * weight
  | RGB of int * int * int
  | Gray of int


We'll also represent this more complicated color space as a variant, but this time, the different tags will have arguments that describe the data available in each case. Note that variants can have multiple arguments, which are separated by *s:

In [20]:
 [RGB (250,70,70); Basic (Green, Regular)];;

- : color list = [RGB (250, 70, 70); Basic (Green, Regular)]


In [21]:
type color = 
|Basic of basic_color
|Bold of basic_color
| RGB of int * int * int
|Gray of int

type color =
    Basic of basic_color
  | Bold of basic_color
  | RGB of int * int * int
  | Gray of int


In [23]:
let color_to_int  = function
|Basic (basic_color) -> basic_color_to_int basic_color
|RGB (r,g,b) -> 16 + b + g * 6 + r *36
|Gray i->232+ i

File "[23]", line 1, characters 20-138:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Bold _


val color_to_int : color -> int = <fun>


我们已经看到，类型错误指出了需要修正哪些方面来完成重构的代码。这非常有用，不过，要让它正确、可靠地发挥作用，需要采用一种合适的方式编写代码、尽可能的增加编译器百年关注你发现`bug`的机会。为了达到这个目的，有一个有用的经验：要避免模式匹配中的`catch-all`全包情况。

In [3]:
module Log_entry = struct
type t = 
{
session_id: string;
time: Time.t;
important: bool;
message: string;
}
end
;;

module Log_entry :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      important : bool;
      message : string;
    }
  end


In [6]:
module Log_entry = struct
type t = 
{
session_id: string;
time: Time.t;
important: bool;
message: string;
}
end
module Heartbeat = struct
type t = 
{
session_id: string;
time: Time.t;
user:string;
credentials: string;
}
end

module Logon = struct
type t = {
session_id: string;
time: Time.t;
user:string;
credentials: string;
}
end

module Log_entry :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      important : bool;
      message : string;
    }
  end


module Heartbeat :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      user : string;
      credentials : string;
    }
  end


module Logon :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      user : string;
      credentials : string;
    }
  end


client_message是一个`Logon`，或是一个`Log_entry`，如果我们像编写采用通用方式处理消息的代码，而不是特定于某一个固定消息类型，就需要使用类似`client_message`的类型作为洞中个不同消息的一个“总体”类型，然后可以匹配`client_message`来确定所处理的特定消息的类型。

In [ ]:
let messages_for_user user messages = 
let (user_messages,_) = 
List.fold messages ~init:([], String.Set.empty)
~f:(fun ((message, user_sessions) as acc) message ->
match message with
|Logon m -> 
if m.Logon.user = user then
(message::messages, Set.add user_sessions) as acc) message ->
match message with
|Logon m ->
if m.Logon.user = user then
(message::messages, Set.add user_sessions m.Logon.session_id)
else acc
|Heartbeat _ | Log_entry _ ->
let session_id = match message with


)